In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
import torch
import robust_value_approx.train_value as train_value

In [ ]:
import double_integrator_utils
vf = double_integrator_utils.get_value_function()

In [ ]:
x0_lo = -1 * torch.ones(vf.sys.x_dim, dtype=vf.dtype)
x0_up = 1 * torch.ones(vf.sys.x_dim, dtype=vf.dtype)

In [ ]:
x_samples_file = '../data/learn_value_function_x'
v_samples_file = '../data/learn_value_function_v'

In [ ]:
num_breaks_train = [3] * vf.sys.x_dim
x_samples_train, v_samples_train = vf.get_value_sample_grid(x0_lo, x0_up, num_breaks_train, update_progress=True)

In [ ]:
torch.save(x_samples_train, x_samples_file + '_train.pt')
torch.save(v_samples_train, v_samples_file + '_train.pt')

In [ ]:
x_samples_train = torch.load(x_samples_file + '_train.pt')
v_samples_train = torch.load(v_samples_file + '_train.pt')

In [ ]:
num_breaks_validation = [100] * vf.sys.x_dim
x_samples_validation, v_samples_validation = vf.get_value_sample_grid(x0_lo, x0_up, num_breaks_validation, update_progress=True)

In [ ]:
torch.save(x_samples_validation, x_samples_file + '_validation.pt')
torch.save(v_samples_validation, v_samples_file + '_validation.pt')

In [ ]:
x_samples_validation = torch.load(x_samples_file + '_validation.pt')
v_samples_validation = torch.load(v_samples_file + '_validation.pt')

In [ ]:
adv = train_value.AdversarialWithBaseline(vf, x0_lo, x0_up,
                                          x_samples_train, v_samples_train,
                                          x_samples_validation, v_samples_validation)
train_opt = train_value.AdversarialWithBaselineTrainingOptions()

In [ ]:
adv.train(train_opt)

In [ ]:
torch.save(adv.baseline_model, '../data/baseline_model.pt')
torch.save(adv.robust_model, '../data/robust_model.pt')